In [1]:
import pandas as pd
import numpy as np
import os
import nltk
import re
import spacy

nlp=spacy.load("en_core_web_lg")

In [2]:
df = pd.read_excel("D:/NLP/data/bank_a_sentiment_analysis_translated_inputdata.xlsx")
df_test=pd.read_excel("D:/NLP/data/bank_a_sentiment_analysis_translated_inputdata_unseen.xlsx")
print(df.info(),"Shape:",df.shape)
df = df[["translated_review","actual_sentiment"]]
df_test = df_test[["translated_review","actual_sentiment"]]
# df=df[df["actual_sentiment"]!="Neutral"]
# df_test=df_test[df_test["actual_sentiment"]!="Neutral"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3486 entries, 0 to 3485
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   review_id          3486 non-null   float64
 1   translated_review  3481 non-null   object 
 2   actual_sentiment   3482 non-null   object 
dtypes: float64(1), object(2)
memory usage: 81.8+ KB
None Shape: (3486, 3)


In [3]:
# Since we cannot impute text data. Therefore, removing empty/blank/NA values from dataset.
df=df.dropna().reset_index()
df=df.drop("index",axis=1)
df_test=df_test.dropna().reset_index()
df_test=df_test.drop("index",axis=1)

In [4]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)

    return "".join(cleantext)

df['preprocessed_text']=df['translated_review'].map(lambda s:preprocess(s)) 
df_test['preprocessed_text']=df_test['translated_review'].map(lambda s:preprocess(s)) 

In [5]:
df['preprocessed_text'] = df['preprocessed_text'].str.replace('[^\w\s]','')
df_test['preprocessed_text'] = df_test['preprocessed_text'].str.replace('[^\w\s]','')

C:\Users\SAURAB~1.MIS\AppData\Local\Temp/ipykernel_26432/4154870127.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['preprocessed_text'] = df['preprocessed_text'].str.replace('[^\w\s]','')
C:\Users\SAURAB~1.MIS\AppData\Local\Temp/ipykernel_26432/4154870127.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['preprocessed_text'] = df_test['preprocessed_text'].str.replace('[^\w\s]','')


In [6]:
# StopWords Removal
from nltk.corpus import stopwords
# stopwords = en.Defaults.stop_words
stop = stopwords.words('english')
not_stop=["no","not","too"]
for i in not_stop:
    if i in stop:
        stop.remove(i)
df['clean_text'] = df['preprocessed_text'].apply(lambda x: " ".join(i for i in x.split() if i not in stop))
# Stemming
df["cleaned_text"]=df["clean_text"].apply(lambda x: " ".join(token.lemma_ for token in nlp(x) if not (token.like_num or token.is_punct or token.is_space or len(token)==1)))
df=df[["cleaned_text","actual_sentiment"]]

df_test['clean_text'] = df_test['preprocessed_text'].apply(lambda x: " ".join(i for i in x.split() if i not in stop))
# Stemming
df_test["cleaned_text"]=df_test["clean_text"].apply(lambda x: " ".join(token.lemma_ for token in nlp(x) if not (token.like_num or token.is_punct or token.is_space or len(token)==1)))
df_test=df_test[["cleaned_text","actual_sentiment"]]
df

,cleaned_text,actual_sentiment
0,branch san marco evangelista employee include ...,Positive
1,commission withdraw money counter theft,Negative
2,f24 app thank,Neutral
3,low commission,Neutral
4,promote support small business territory,Positive
...,...,...
3472,availability rapidity,Neutral
3473,serious reliable comfort payment etc,Neutral
3474,norm,Neutral
3475,poor interest customer,Negative


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

def topic(df):
    tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
    dtm = tfidf.fit_transform(df['cleaned_text'])
    nmf_model = NMF(n_components=3,random_state=42)
    # This can take awhile, we're dealing with a large amount of documents!
    nmf_model.fit(dtm)
    for index,topic in enumerate(nmf_model.components_):
        print(f'THE TOP 15 WORDS FOR TOPIC #{index+1}')
        print([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
        print('\n')

topic(df[df["actual_sentiment"]=="Positive"])

THE TOP 15 WORDS FOR TOPIC #1
['problem', 'banking', 'branch', 'helpful', 'customer', 'competent', 'feel', 'online', 'kind', 'efficient', 'staff', 'available', 'good', 'excellent', 'service']


THE TOP 15 WORDS FOR TOPIC #2
['professionalit', 'cordiality', 'clarity', 'customer', 'professional', 'reliability', 'employee', 'seriousness', 'staff', 'professionalism', 'efficiency', 'competence', 'kindness', 'courtesy', 'availability']


THE TOP 15 WORDS FOR TOPIC #3
['close', 'problem', 'safe', 'thank', 'employee', 'fine', 'consider', 'people', 'need', 'excellent', 'good', 'year', 'customer', 'reliable', 'bank']




In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

def topic(df):
    tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
    dtm = tfidf.fit_transform(df['cleaned_text'])
    nmf_model = NMF(n_components=7,random_state=42)
    # This can take awhile, we're dealing with a large amount of documents!
    nmf_model.fit(dtm)
    for index,topic in enumerate(nmf_model.components_):
        print(f'THE TOP 15 WORDS FOR TOPIC #{index+1}')
        print([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
        print('\n')

topic(df[df["actual_sentiment"]=="Negative"])
    

THE TOP 15 WORDS FOR TOPIC #1
['bad', 'offer', 'free', 'change', 'different', 'reliable', 'time', 'need', 'condition', 'account', 'recommend', 'make', 'compare', 'like', 'bank']


THE TOP 15 WORDS FOR TOPIC #2
['compare', 'card', 'average', 'operation', 'transfer', 'expense', 'condition', 'commission', 'online', 'low', 'increase', 'account', 'management', 'high', 'cost']


THE TOP 15 WORDS FOR TOPIC #3
['closure', 'change', 'problem', 'reject', 'remove', 'depend', 'long', 'hour', 'ask', 'year', 'opening', 'inadequate', 'atm', 'work', 'branch']


THE TOP 15 WORDS FOR TOPIC #4
['employee', 'forget', 'despite', 'attention', 'relationship', 'need', 'feel', 'attentive', 'year', 'poor', 'small', 'assist', 'follow', 'available', 'customer']


THE TOP 15 WORDS FOR TOPIC #5
['great', 'employee', 'clear', 'lot', 'feel', 'investment', 'worsen', 'dear', 'long', 'bad', 'poor', 'online', 'offer', 'good', 'service']


THE TOP 15 WORDS FOR TOPIC #6
['old', 'previous', 'payment', 'expense', 'problem', 

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = tfidf.fit_transform(df['cleaned_text'])
dtm

<3477x1139 sparse matrix of type '<class 'numpy.float64'>'
	with 15131 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.decomposition import NMF

nmf_model = NMF(n_components=7,random_state=42)
# This can take awhile, we're dealing with a large amount of documents!
nmf_model.fit(dtm)

NMF(n_components=7, random_state=42)

In [13]:
print("Number of features:",len(tfidf.get_feature_names_out()))

import random

for i in range(10):
    random_word_id = random.randint(0,len(tfidf.get_feature_names_out()))
    print(tfidf.get_feature_names_out()[random_word_id])

Number of features: 1139
intend
happy
family
moment
feel
office
indecent
milan
expensive
effectively


In [16]:
for i in range(10):
    random_word_id = random.randint(0,1139)
    print(tfidf.get_feature_names_out()[random_word_id])

7
currently
city
adequate
unicredit
dirty
willingness
cashier
marina
cordialit
pro


In [17]:
print(len(nmf_model.components_))
nmf_model.components_

7


array([[0.00000000e+00, 7.42937892e-03, 4.38414819e-03, ...,
        5.04364088e-05, 1.36242031e-05, 0.00000000e+00],
       [7.65556302e-03, 0.00000000e+00, 9.57278786e-05, ...,
        9.25257070e-03, 3.08580508e-05, 6.00033817e-04],
       [5.63249968e-03, 2.46890182e-02, 4.59104084e-03, ...,
        1.47801616e-03, 2.11728183e-03, 5.90738632e-02],
       ...,
       [6.17071655e-03, 1.17365926e-02, 0.00000000e+00, ...,
        1.08678393e-03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 3.76979067e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.66376834e-04, 0.00000000e+00, 5.50584026e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [19]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['account', 'satisfied', 'simple', 'customer', 'quality', 'efficiency', 'internet', 'use', 'counter', 'banking', 'cost', 'offer', 'online', 'excellent', 'service']


THE TOP 15 WORDS FOR TOPIC #1
['professionalit', 'professional', 'reliability', 'customer', 'clarity', 'employee', 'cordiality', 'seriousness', 'professionalism', 'staff', 'efficiency', 'competence', 'kindness', 'courtesy', 'availability']


THE TOP 15 WORDS FOR TOPIC #2
['trust', 'cost', 'people', 'know', 'close', 'fine', 'work', 'consider', 'like', 'account', 'need', 'excellent', 'year', 'customer', 'bank']


THE TOP 15 WORDS FOR TOPIC #3
['employee', 'people', 'courteous', 'prepared', 'solve', 'need', 'branch', 'problem', 'customer', 'professional', 'helpful', 'competent', 'kind', 'staff', 'available']


THE TOP 15 WORDS FOR TOPIC #4
['internet', 'web', 'human', 'problem', 'continue', 'personal', 'employee', 'banking', 'year', 'service', 'condition', 'customer', 'relationship', 'feel', 'goo